# On the natural exponential family generated by the generalised

hyperbolic secant distribution

Karim AI (University of Bath)

# Intro

This vignette serves two purposes

-   Introduced the missing GLM distribution
-   review of GLMs and extensions in `R`

# Preliminaries

A continuous real random variable $Z$ is said to follow a *generalised hyperbolic secant distribution* (GHS) ([Harkness and Harkness 1968](#ref-Harkness_GHS)) with parameter $\lambda>0$ if the corresponding probability density function can be written as:

$$
g(z|\lambda)
=
\frac{2^{\lambda-2}\,\Gamma\left(\frac{\lambda}{2}+i\frac{z}{2}\right)\Gamma\left(\frac{\lambda}{2}-i\frac{z}{2}\right)}{\pi\,\Gamma(\lambda)}\,,\quad z \in {\mathcal R}\,,
$$

where $\Gamma(w)$ is the [Gamma function](https://en.wikipedia.org/wiki/Gamma_function) in the complex plane, that is

$$
\Gamma(w)=\int_{0}^\infty t^{w-1}\exp(-t)dt\,,\qquad Re(w)>0\,.
$$

The name of the distribution comes from the case $\lambda=1$, e.g.

$$
g(z|1)
=
\frac{\Gamma\left(\frac{1}{2}+i\frac{z}{2}\right)\Gamma\left(\frac{1}{2}-i\frac{z}{2}\right)}{2\pi\Gamma(1)}
=\frac{\pi}{2\pi\,\cosh(\pi\,z/2)}=\frac{1}{2}\textrm{sech}\left(\frac{\pi\,z}{2}\right)
$$

which can be obtained by using identity 6.1.30 in Abramowitz and Stegun ([1964](#ref-abramowitz_stegun)).

Using the properties of the complex Gamma function (Section 6.1 in Abramowitz and Stegun ([1964](#ref-abramowitz_stegun))) we can write the log densities as follows:

$$
\log g(z|\lambda)
=
(\lambda-2)\log(2)-\log(\pi)-\log \Gamma(\lambda)+
2\,Re\left(\log \Gamma\left(\frac{\lambda}{2}+i\frac{z}{2}\right)\right)
$$ To compute $g(z|\lambda)$ we only need a function to evaluate the log of the Gamma function in the complex plane and here we use the `gls` package (Hankin ([2006](#ref-gsl_package))) to do so.

In [ ]:
library(gsl)
dghs  <- function(z,lambda,log=F){
  logf  <- (lambda-2)*log(2) - log(pi) - 
    lgamma(lambda) + 2*Re(lngamma_complex((lambda+z*1i)/2))
  if (log){
    res <- logf
  }else{
    res <- exp(logf) 
  }
  res
}

In [ ]:
N_grid <- 1000
z_grid <- seq(-5,5,length=N_grid)
f1 <- dghs(z_grid,lambda=1,log=F)
f2 <- dghs(z_grid,lambda=2,log=F)
f4 <- dghs(z_grid,lambda=4,log=F)
f8 <- dghs(z_grid,lambda=8,log=F)
plot(z_grid,f1,type="l",
     xlab="z", 
     ylab="density")
lines(z_grid,f2,lty=2)
lines(z_grid,f4,lty=3)
lines(z_grid,f8,lty=4)
abline(v=0,col="grey",lwd=0.5)
legend("topright",lty=c(1,2,3,4),legend=c(expression(lambda == 1),
                                          expression(lambda == 2),
                                          expression(lambda == 4),
                                          expression(lambda == 8)))

The densities $g(z|\lambda)$ are symmetric about zero (see <a href="#fig-dens-ghs" class="quarto-xref">Figure 1</a>) with exponentially decaying tails.

The moment generating function of the GHS distribution is well defined and is given by:

The above implies the GHS distribution is infinitely divisible so that the probability distribution of independent sums of GHS distributions is also a GHS distribution. Then any GHS distribution with integer $\lambda$ can be reproduced by convolutions of the basic density with $\lambda=1$.

The distribution has zero mean and the parameter $\lambda$ that equals the variance.

In this way, the *standard* distribution is $g(z|1)$. The natural exponential family (NEF) generated by the GHS distribution has therefore densities of the form:

$$
\tilde{f}_Z(z|\theta,\lambda)=\exp(\theta\,z-\lambda\log(\sec(\theta)))\,g(z|\lambda)
\,,\qquad |\theta|<\frac{\pi}{2}\,.
$$

By making the transformation $Y=Z/\lambda$ we obtain the so-called *exponential family* form (see McCullagh and Nelder ([1989](#ref-McCullagh_Nelder_1989)))

<span id="eq-nef-ghs-dens1">$$
\tilde{f}_Y(y|\theta,\lambda)=\lambda\,\tilde{f}_Z(\lambda\,y|\theta,\lambda)=
\lambda\,g(\lambda\,y|\lambda)\,
\exp\left(\frac{\theta\,y-\psi(\theta)}{1/\lambda}\right)
 \qquad(1)$$</span>

where $\psi(\theta)=\log(\sec(\theta))$. This is a *reproductive exponential dispersion model* in the language of Jorgensen ([1997](#ref-jorgensen_1997)) and $\phi=1/\lambda$ is the so-called dispersion parameter. Clearly, if $\lambda$ is unknown then $\tilde{f}_Y(y|\theta,\lambda)$ is not a two-dimensional exponential family.

We will follow Morris ([1982](#ref-morris_1982_1)) and call the family $\tilde{f}_Y(y|\theta,\lambda)$ the NEF-GHS. The mean and variance of the NEF-GHS are given by:

We can reparametrise in terms of the mean $\mu$ and the dispersion parameter $\phi=1/\lambda$ as follows

$$
f(y|\mu,\phi)=\tilde{f}(y|\arctan(\mu),1/\phi)=
\frac{1}{\psi}\,g(y/\psi|1/\psi)\,
\exp\left(\lambda\left[y\,\arctan(\mu)-\frac{1}{2}\log(1+\mu^2)\right]\right)
$$

Form the above we obtain that $g(\mu)=\arctan(\mu)$ is the corresponding canonical link of the family.

In [ ]:
   my_family <- list(
       family   = "my_family",
       link     = "arctan",  
       linkfun  = function(mu)atan(mu), # canonical link function
       linkinv  = function(eta)tan(eta), # inverse link function
       variance = function(mu) 1+mu^2,
       mu.eta   = function(eta) sec(eta)^2, # Derivative of mean wrt eta
       dev.resids = function(y, mu,wt){2*wt*y(atan(y)-atan(mu))+log((1+mu^2)/(1+y^2))}, # Deviance residuals
       aic = function(y, mu, weights, data) 1, # AIC
       validmu = function(mu) is.finite(mu) , # Valid mu
       valideta = function(eta) abs(eta)<pi/2
   )
   class(my_family) <- "family"

In [ ]:
 nef_ghs<-gaussian()


nef_ghs$family = "nef ghs"
nef_ghs$link     = "arctan"
nef_ghs$linkfun  = function(mu)atan(mu) # canonical link function
nef_ghs$linkinv  = function(eta)tan(eta) # inverse link function
nef_ghs$variance = function(mu) 1+mu^2
nef_ghs$mu.eta   = function(eta) sec(eta)^2 # Derivative of mean wrt eta
nef_ghs$dev.resids = function(y, mu,wt){2*wt*y(atan(y)-atan(mu))+log((1+mu^2)/(1+y^2))} # Deviance residuals
nef_ghs$aic = function(y, mu, weights, data) 1 # AIC
nef_ghs$validmu = function(mu) is.finite(mu)  # Valid mu
nef_ghs$valideta = function(eta) abs(eta)<pi/2
nef_ghs$dispersion = 1

str(nef_ghs)

List of 12
 $ family    : chr "nef ghs"
 $ link      : chr "arctan"
 $ linkfun   :function (mu)  
 $ linkinv   :function (eta)  
 $ variance  :function (mu)  
 $ dev.resids:function (y, mu, wt)  
 $ aic       :function (y, mu, weights, data)  
 $ mu.eta    :function (eta)  
 $ initialize:  expression({  n <- rep.int(1, nobs)  if (is.null(etastart) && is.null(start) && is.null(mustart) && ((family$link| __truncated__
 $ validmu   :function (mu)  
 $ valideta  :function (eta)  
 $ dispersion: num 1
 - attr(*, "class")= chr "family"

In [ ]:
y1<-rnorm(100,4,1)
y2<-rnorm(100,6,1)
y<-c(y1,y2)
x<-c(rep(0,100),rep(1,100))
df <-data.frame(x=x,y=y)

lin_mod<-lm( y ~ x, data = df)

summary(lin_mod)$coeff

            Estimate Std. Error  t value     Pr(>|t|)
(Intercept) 3.995151  0.1022565 39.06992 5.254096e-95
x           1.946322  0.1446125 13.45888 9.542022e-30

# Parameter interpretation

all in MOrris the mean measures symmetry with limits $\mu=0$ is symmetric and $|\mu|\to \infty$ is a gamma, see also page 103 of Jorgensen

$\lambda \to \infty$ also converges to normal

when $\mu=0$ the distribution is symmeytric and tails are exponential

for postive $\theta$ the riight tail is the same as an specific gamma (Morris end of section 5)

# likelihood inference

$$
\ell(\beta,\phi)
$$

Abramowitz, Milton, and Irene A. Stegun. 1964. *Handbook of Mathematical Functions with Formulas, Graphs, and Mathematical Tables*. Ninth Dover printing, tenth GPO printing. New York City: Dover.

Hankin, Robin K. S. 2006. “Special Functions in r: Introducing the Gsl Package.” *R News* 6.

Harkness, W. L., and M. L. Harkness. 1968. “Generalized Hyperbolic Secant Distributions.” *Journal of the American Statistical Association* 63 (321): 329–37. <http://www.jstor.org/stable/2283852>.

Jorgensen, B. 1997. *The Theory of Dispersion Models*. Chapman & Hall/CRC Monographs on Statistics & Applied Probability. Taylor & Francis. <https://books.google.co.uk/books?id=0gO7bgs_eSYC>.

McCullagh, P., and J. A. Nelder. 1989. *Generalized Linear Models*. London: Chapman & Hall / CRC.

Morris, Carl N. 1982. “<span class="nocase">Natural Exponential Families with Quadratic Variance Functions</span>.” *The Annals of Statistics* 10 (1): 65–80. <https://doi.org/10.1214/aos/1176345690>.